## Collection des images

In [3]:
# !pip install opencv-python

## Importation des Packages

In [4]:
# OpenCV
import cv2

# uuid
import uuid

## Opérations sur le Système d'exploitation
import os

# la gestion du temps
import time

## Définition des images à collecter

In [5]:
labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
number_images = 5

## Configuration du dossier des images

In [11]:
IMAGES_PATH = os.path.join('.', 'tensorflow', 'workspace', 'images', 'collectedimages')

In [12]:
IMAGES_PATH

'.\\tensorflow\\workspace\\images\\collectedimages'

In [13]:
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir - {IMAGES_PATH}
    if os.name == 'nt':
        !mkdir {IMAGES_PATH}
        
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}